In [ ]:
import pandas as pd
import folium
import requests
import json
import numpy as np

# 1. 설정
pkl_path = r"C:\Users\playdata2\seohyun군승합승용.pkl"
# 대상 연도 리스트 설정
target_years = [2022, 2023, 2024] 

# 2. GeoJSON 로드 (전역 변수로 한 번만 로드)
geo_url = 'https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-municipalities-2018-geo.json'
geo_data = requests.get(geo_url).json()

def create_clean_map(path, year):
    # 데이터 로드
    full_df = pd.read_pickle(path)
    
    for v in ['car', 'van']:
        category = "승용차" if v == "car" else "승합차"
        
        # 해당 연도 및 차종 데이터 필터링
        df = full_df[(full_df['reg_year'].astype(str) == str(year)) & (full_df['vehicle_type'] == v)]
        
        # 시군구별 합계 계산
        df_sum = df.groupby('sigungu_name')['car_count'].sum().reset_index()
        
        # 매핑용 사전 생성
        final_mapping = {}
        for feature in geo_data['features']:
            g_name = feature['properties']['name']
            matched = df_sum[df_sum['sigungu_name'].str.contains(g_name, na=False)]
            val = matched['car_count'].sum() if not matched.empty else 0
            
            # 직접 데이터 보정 (필요시 연도별 조건 추가 가능)
            if g_name == "계룡시":
                val = 18600 if v == 'car' else 750 
            elif g_name == "계양구":
                val = 110000 if v == 'car' else 4500
            
            final_mapping[g_name] = int(val)

        # 지도용 데이터프레임 생성
        df_final = pd.DataFrame(list(final_mapping.items()), columns=['name', 'value'])
        df_final['display_val'] = df_final['value'] / 1000

        # --- [색상 극대화: Bins 설정] ---
        series = df_final['display_val']
        quantiles = [0, 0.1, 0.3, 0.5, 0.7, 0.85, 0.95, 1]
        bins = series.quantile(quantiles).unique().tolist()
        if len(bins) < 3: bins = 6

        # 지도 생성
        m = folium.Map(location=[36.5, 127.5], zoom_start=7)

        # 1. 단계구분도 추가 (검은색 얇은 선 적용)
        choropleth = folium.Choropleth(
            geo_data=geo_data,
            data=df_final,
            columns=['name', 'display_val'],
            key_on='feature.properties.name',
            fill_color='YlOrRd', 
            fill_opacity=0.6,
            line_color='black',  # 검은색 선
            line_weight=0.3,     # 얇은 두께
            line_opacity=1,
            bins=bins,           
            legend_name=f'{year}년 {category} 등록수 (천 대)'
        ).add_to(m)

        # 2. 툴팁 추가
        for feature in choropleth.geojson.data['features']:
            name = feature['properties']['name']
            real_val = final_mapping.get(name, 0)
            feature['properties']['tooltip_text'] = f"{name}: {real_val:,} 대"

        choropleth.geojson.add_child(
            folium.GeoJsonTooltip(fields=['tooltip_text'], aliases=['🚗 현황:'], labels=False)
        )

        # 3. 출력 처리
        print(f"✅ {year}년 {category} 지도 생성 완료")
        
        try:
            from IPython.display import display
            display(m)
        except ImportError:
            filename = f"map_{year}_{v}.html"
            m.save(filename)
            print(f"🌐 지도가 {filename}으로 저장되었습니다.")

# --- 실행 부분: 반복문을 통해 여러 연도 처리 ---
if __name__ == "__main__":
    for y in target_years:
        create_clean_map(pkl_path, y)